In [1]:
# importamos las librerías con las que vamos a trabajar

# Trabajar con bases de datos y python
# -----------------------------------------------------------------------
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors


# Trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
df_jugadores=pd.read_csv(f'../../data/transformed/stats_jugadores_transformed.csv')
df_info=pd.read_csv(f'../../data/transformed/info_clubes_transformed.csv')
df_clasidicacion=pd.read_csv(f'../../data/transformed/clasificaciones_premier_transformed.csv')
df_estadios=pd.read_csv(f'../../data/transformed/asistencia_estadios_transformed.csv')
df_clubes=pd.read_csv(f'../../data/transformed/clubes.csv')

In [3]:
try:
    conexionpremier=psycopg2.connect(
    database="premier",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432")
except OperationalError as e:
    if e.pgcode ==errorcodes.INVALID_PASSWORD:
        print("La constraseña es incorrecta")
    elif e.pgcode ==errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión")

In [4]:
# Primero, deshacer la transacción fallida
conexionpremier.rollback()

cursorpremier = conexionpremier.cursor()
conexionpremier

<connection object at 0x000002A80C8E3450; dsn: 'user=postgres password=xxx dbname=premier host=localhost port=5432', closed: 0>

In [5]:
# Tabla principal: Clubes con id_equipo como PRIMARY KEY
cursorpremier.execute("""
    CREATE TABLE IF NOT EXISTS clubes (
        id_equipo SERIAL PRIMARY KEY,
        equipo VARCHAR(255) UNIQUE
    );
""")

# Tabla 1: Asistencia Estadios
cursorpremier.execute("""
    CREATE TABLE IF NOT EXISTS asistencia_estadios (
        estadio VARCHAR(255),
        id_equipo INT,
        capacidad INT,
        espectadores INT,
        promedio INT,
        partidos INT,
        lleno FLOAT,
        maximo_rendimiento FLOAT,
        temporada INT,
        PRIMARY KEY (id_equipo, temporada),
        FOREIGN KEY (id_equipo) REFERENCES clubes(id_equipo) ON DELETE CASCADE
    );
""")

# Tabla 2: Clasificaciones Premier
cursorpremier.execute("""
    CREATE TABLE IF NOT EXISTS clasificaciones_premier (
        temporada INT,
        posicion INT,
        id_equipo INT,
        goles_favor_total FLOAT,
        goles_contra_total FLOAT,
        puntos_total INT,
        goles_favor_local FLOAT,
        goles_contra_local FLOAT,
        puntos_local INT,
        goles_favor_visitante FLOAT,
        goles_contra_visitante FLOAT,
        puntos_visitante INT,
        PRIMARY KEY (id_equipo, temporada),
        FOREIGN KEY (id_equipo) REFERENCES clubes(id_equipo) ON DELETE CASCADE
    );
""")

# Tabla 3: Info Clubes
cursorpremier.execute("""
    CREATE TABLE IF NOT EXISTS info_clubes (
        id_equipo INT PRIMARY KEY,
        nombre_completo VARCHAR(255),
        ano_fundacion INT,
        pais VARCHAR(255),
        confederacion_nacional VARCHAR(255),
        colores_principales VARCHAR(255),
        colores_alternativos VARCHAR(255),
        apodo VARCHAR(255),
        web_oficial VARCHAR(255),
        titulos_internacionales INT,
        copa_mundial_clubes INT,
        uefa_europa_league INT,
        supercopa_europea INT,
        recopa_de_europa INT,
        titulos_nacionales INT,
        liga_inglesa INT,
        fa_cup INT,
        supercopa_inglesa INT,
        copa_liga_inglesa INT,
        full_members_cup INT,
        FOREIGN KEY (id_equipo) REFERENCES clubes(id_equipo) ON DELETE CASCADE
    );
""")

# Tabla 4: Stats Jugadores
cursorpremier.execute("""
    CREATE TABLE IF NOT EXISTS stats_jugadores (
        temporada INT,
        id_jugador INT,
        nombre_jugador VARCHAR(255),
        edad INT,
        nacionalidad VARCHAR(50),
        id_equipo INT,
        partidos_jugados INT,
        minutos INT,
        goles INT,
        tarjetas_amarillas INT,
        tarjetas_rojas INT,
        PRIMARY KEY (id_jugador, temporada),
        FOREIGN KEY (id_equipo) REFERENCES clubes(id_equipo) ON DELETE CASCADE
    );
""")

# Confirmación de la creación de tablas
conexionpremier.commit()
print("Tablas creadas exitosamente en la base de datos.")

Tablas creadas exitosamente en la base de datos.


In [6]:
df_jugadores=pd.read_csv(f'../../data/transformed/stats_jugadores_transformed.csv')
df_info=pd.read_csv(f'../../data/transformed/info_clubes_transformed.csv')
df_clasidicacion=pd.read_csv(f'../../data/transformed/clasificaciones_premier_transformed.csv')
df_estadios=pd.read_csv(f'../../data/transformed/asistencia_estadios_transformed.csv')
df_clubes=pd.read_csv(f'../../data/transformed/clubes.csv')

In [7]:
# Primero, deshacer la transacción fallida
conexionpremier.rollback()

cursorpremier = conexionpremier.cursor()
conexionpremier

<connection object at 0x000002A80C8E3450; dsn: 'user=postgres password=xxx dbname=premier host=localhost port=5432', closed: 0>

Insertar valores

In [8]:
try:
    cursorpremier = conexionpremier.cursor()

    # Paso 1: Insertar equipos en 'clubes' y construir el mapeo id_equipo sin duplicados
    equipo_id_map = {}
    for equipo in df_clubes['Equipos']:
        # Verificar si el equipo ya existe
        cursorpremier.execute("SELECT id_equipo FROM clubes WHERE equipo = %s;", (equipo,))
        result = cursorpremier.fetchone()
        
        if result:
            # Si el equipo ya existe, obtener el id_equipo existente
            id_equipo = result[0]
        else:
            # Si el equipo no existe, insertar y obtener el nuevo id_equipo
            cursorpremier.execute("INSERT INTO clubes (equipo) VALUES (%s) RETURNING id_equipo;", (equipo,))
            id_equipo = cursorpremier.fetchone()[0]
        
        equipo_id_map[equipo] = id_equipo

    # Paso 2: Actualizar las listas de tuplas usando el mapeo de 'id_equipo'
    asistencia_estadios_data = [
        (
            row['Estadio'], equipo_id_map[row['Equipo']], row['Capacidad'], row['Espectadores'],
            row['Promedio'], row['Partidos'], row['Lleno'], row['Máximo rendimiento'], row['Temporada']
        )
        for _, row in df_estadios.iterrows()
    ]

    clasificaciones_premier_data = [
        (
            row['temporada'], row['posicion'], equipo_id_map[row['equipo']], row['goles_favor_total'],
            row['goles_contra_total'], row['puntos_total'], row['goles_favor_local'],
            row['goles_contra_local'], row['puntos_local'], row['goles_favor_visitante'],
            row['goles_contra_visitante'], row['puntos_visitante']
        )
        for _, row in df_clasidicacion.iterrows()
    ]

    info_clubes_data = [
        (
            equipo_id_map[row['Equipo']], row['Nombre Completo'], row['Año de fundación'], row['País'],
            row['Confederación Nacional'], row['Colores principales'], row['Colores alternativos'],
            row['Apodo'], row['Web oficial'], row['Títulos Internacionales'], row['Copa Mundial de Clubes'],
            row['UEFA Europa League'], row['Supercopa Europea'], row['Recopa de Europa'], row['Títulos Nacionales'],
            row['Liga Inglesa'], row['FA Cup'], row['Supercopa Inglesa'], row['Copa de la Liga de Inglaterra'],
            row["Full Members' Cup"]
        )
        for _, row in df_info.iterrows()
    ]

    stats_jugadores_data = [
        (
            row['temporada'], row['id_jugador'], row['nombre_jugador'], row['edad'], row['nacionalidad'],
            equipo_id_map[row['equipo']], 
            row['partidos_jugados'], row['minutos'], row['goles'],
            row['tarjetas_amarillas'], row['tarjetas_rojas']
        )
        for _, row in df_jugadores.iterrows()
    ]

    # Paso 3: Insertar datos en las tablas restantes
    cursorpremier.executemany("""
        INSERT INTO asistencia_estadios (estadio, id_equipo, capacidad, espectadores, promedio, partidos, lleno, maximo_rendimiento, temporada)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, asistencia_estadios_data)

    cursorpremier.executemany("""
        INSERT INTO clasificaciones_premier (temporada, posicion, id_equipo, goles_favor_total, goles_contra_total, puntos_total, goles_favor_local, goles_contra_local, puntos_local, goles_favor_visitante, goles_contra_visitante, puntos_visitante)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, clasificaciones_premier_data)

    cursorpremier.executemany("""
        INSERT INTO info_clubes (id_equipo, nombre_completo, ano_fundacion, pais, confederacion_nacional, colores_principales, colores_alternativos, apodo, web_oficial, titulos_internacionales, copa_mundial_clubes, uefa_europa_league, supercopa_europea, recopa_de_europa, titulos_nacionales, liga_inglesa, fa_cup, supercopa_inglesa, copa_liga_inglesa, full_members_cup)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, info_clubes_data)

    cursorpremier.executemany("""
        INSERT INTO stats_jugadores (temporada, id_jugador, nombre_jugador, edad, nacionalidad, id_equipo, partidos_jugados, minutos, goles, tarjetas_amarillas, tarjetas_rojas)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, stats_jugadores_data)

    # Confirmar transacciones
    conexionpremier.commit()
    print("Datos insertados exitosamente en la base de datos.")

except Exception as e:
    print(f"Ocurrió un error: {e}")
    conexionpremier.rollback()

Datos insertados exitosamente en la base de datos.
